In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## Door knob

In [4]:
from pkg.utils.code_scraps import *

In [5]:
gscene.show_pose(crob.home_pose)

In [6]:
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(0.5,0.5,0.5), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=True, collision=True, color=(1,0,0,1))

In [9]:
lever_s, knob_s = add_lever(pscene, knob)

### prepare planner

In [10]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [11]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

### Plan

In [12]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['knob', 'lever']
('r1', 'knob')


In [13]:
goal_nodes = [('r2','knob')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

try: 0 - ('r1', 'knob')->('r2', 'knob')
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r1', 'grip0')
try transition motion
transition motion tried: True
result: 0 - ('r1', 'knob')->('r1', 'grip0') = success
branching: 0->1 (0.14/30.0 s, steps/err: 57(100.234031677 ms)/0.000989529519831)
try: 1 - ('r1', 'grip0')->('r2', 'grip0')
end
constrained motion tried: True
result: 1 - ('r1', 'grip0')->('r2', 'grip0') = success
branching: 1->2 (0.54/30.0 s, steps/err: 80(395.957946777 ms)/0.000101350640859)
try: 2 - ('r2', 'grip0')->('r2', 'knob')
Goal reached
result: 2 - ('r2', 'grip0')->('r2', 'knob') = success
branching: 2->3 (0.54/30.0 s, steps/err: 1(3.72886657715 ms)/0)
++ adding return motion to acquired answer ++
Goal reached
try: 1 - ('r1', 'grip0')->('r2', 'grip0')
end
constrained motion tried: True
result: 1 - ('r1', 'grip0')->('r2', 'grip0') = success
branching: 1->5 

In [14]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

('r1', 'knob')->('r1', 'grip0')
('r1', 'grip0')->('r2', 'grip0')
('r2', 'grip0')->('r2', 'knob')
('r2', 'knob')->('r2', 'knob')


### push back

In [15]:
term_state = snode_schedule[-1].state
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

['knob', 'lever']
('r2', 'knob')


In [22]:
goal_nodes = [("r1",'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=False)

try: 0 - ('r2', 'knob')->('r1', 'knob')
result: 0 - ('r2', 'knob')->('r1', 'knob') = fail
try: 0 - ('r2', 'knob')->('r1', 'knob')
result: 0 - ('r2', 'knob')->('r1', 'knob') = fail
try: 0 - ('r2', 'knob')->('r1', 'knob')
result: 0 - ('r2', 'knob')->('r1', 'knob') = fail
try: 0 - ('r2', 'knob')->('r2', 'grip0')
try transition motion
transition motion tried: True
result: 0 - ('r2', 'knob')->('r2', 'grip0') = success
branching: 0->1 (0.2/30.0 s, steps/err: 66(155.012130737 ms)/0.00135562799192)
try: 0 - ('r2', 'knob')->('r2', 'grip0')
try transition motion
transition motion tried: True
result: 0 - ('r2', 'knob')->('r2', 'grip0') = success
branching: 0->2 (0.3/30.0 s, steps/err: 62(95.5100059509 ms)/0.00182918461865)
try: 1 - ('r2', 'grip0')->('r1', 'grip0')
end
constrained motion tried: True
result: 1 - ('r2', 'grip0')->('r1', 'grip0') = success
branching: 1->3 (0.66/30.0 s, steps/err: 80(365.427970886 ms)/1.81064113522e-05)
try: 3 - ('r1', 'grip0')->('r1', 'knob')
Goal reached
result: 3 -

In [23]:
snode_schedule = tplan.get_best_schedule(home_pose=crob.home_pose)
ppline.play_schedule(snode_schedule)

('r2', 'knob')->('r2', 'grip0')
('r2', 'grip0')->('r1', 'grip0')
('r1', 'grip0')->('r1', 'knob')
('r1', 'knob')->('r1', 'knob')
